# 01 — Exploración de Datos FOREX

Este notebook realiza un **Análisis Exploratorio de Datos (EDA)** sobre los datos de mercado FOREX
que alimentan el sistema de trading. Se cubren:

1. Estructura de los datos OHLCV (Open, High, Low, Close, Volume)
2. Distribuciones de retornos y volatilidad
3. Indicadores técnicos pre-calculados
4. Correlaciones entre pares de divisas
5. Análisis temporal (sesiones de mercado)

> **Datos**: Se utilizan datos históricos de 8 pares FOREX en múltiples temporalidades.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

# Directorio de datos (sample para demo, data/ para dataset completo)
DATA_DIR = Path('../data/sample')
RATES_DIR = DATA_DIR / 'rates'
INDICATORS_DIR = DATA_DIR / 'indicators'

print(f'Directorio de datos: {DATA_DIR}')
print(f'Archivos de rates: {sorted(RATES_DIR.glob("*.csv"))}')
print(f'Archivos de indicadores: {sorted(INDICATORS_DIR.glob("*.csv"))}')

## 1. Carga y estructura de datos OHLCV

Los datos OHLCV representan **velas** (candlesticks) de precio:
- **Open**: Precio de apertura del periodo
- **High**: Precio máximo alcanzado
- **Low**: Precio mínimo alcanzado
- **Close**: Precio de cierre del periodo
- **Volume**: Número de ticks en el periodo

In [ ]:
def load_rates(symbol: str, timeframe: str = 'h1') -> pd.DataFrame:
    """Carga un CSV de rates y parsea fechas."""
    path = RATES_DIR / f'{symbol}_{timeframe}_rates.csv'
    if not path.exists():
        print(f'Archivo no encontrado: {path}')
        return pd.DataFrame()
    df = pd.read_csv(path)
    df['datetime'] = pd.to_datetime(df['readable_date'])
    df = df.set_index('datetime').sort_index()
    return df

# Cargar EURUSD H1 como ejemplo principal
eurusd = load_rates('eurusd', 'h1')
print(f'EURUSD H1: {len(eurusd)} velas')
print(f'Periodo: {eurusd.index.min()} → {eurusd.index.max()}')
print()
eurusd.head()

In [ ]:
# Estadísticas descriptivas
print('=== Estadísticas descriptivas EURUSD H1 ===')
eurusd[['open', 'high', 'low', 'close', 'volume']].describe().round(6)

## 2. Visualización de precios

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [3, 1]})

# Gráfico de precio close
axes[0].plot(eurusd.index, eurusd['close'], color='steelblue', linewidth=0.8)
axes[0].set_title('EURUSD — Precio de Cierre (H1)', fontsize=14)
axes[0].set_ylabel('Precio (USD)')
axes[0].grid(True, alpha=0.3)

# Volumen
axes[1].bar(eurusd.index, eurusd['volume'], color='gray', alpha=0.5, width=0.04)
axes[1].set_title('Volumen (ticks por hora)', fontsize=12)
axes[1].set_ylabel('Ticks')

plt.tight_layout()
plt.show()

## 3. Distribución de retornos

Los retornos logarítmicos permiten analizar la distribución de cambios de precio.
En mercados financieros, los retornos suelen presentar:
- **Leptokurtosis** (colas pesadas — eventos extremos más frecuentes que en una normal)
- **Asimetría** leve
- **Agrupamiento de volatilidad** (periodos de alta/baja volatilidad se agrupan)

In [ ]:
# Calcular retornos logarítmicos
eurusd['log_return'] = np.log(eurusd['close'] / eurusd['close'].shift(1))
returns = eurusd['log_return'].dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma de retornos
axes[0].hist(returns, bins=80, density=True, color='steelblue', alpha=0.7, edgecolor='white')
# Superponer distribución normal
x = np.linspace(returns.min(), returns.max(), 200)
normal_pdf = (1 / (returns.std() * np.sqrt(2*np.pi))) * np.exp(-0.5*((x - returns.mean())/returns.std())**2)
axes[0].plot(x, normal_pdf, 'r--', linewidth=2, label='Normal teórica')
axes[0].set_title('Distribución de Retornos Logarítmicos (H1)', fontsize=12)
axes[0].set_xlabel('Log Return')
axes[0].legend()

# Q-Q plot
from scipy import stats
stats.probplot(returns, dist='norm', plot=axes[1])
axes[1].set_title('Q-Q Plot vs Normal', fontsize=12)

plt.tight_layout()
plt.show()

print(f'Media:     {returns.mean():.6f}')
print(f'Std:       {returns.std():.6f}')
print(f'Skewness:  {returns.skew():.4f}')
print(f'Kurtosis:  {returns.kurtosis():.4f}  (normal = 0, >0 = colas pesadas)')

## 4. Rango intra-vela (High - Low)

El rango de cada vela es una medida de **volatilidad intra-periodo**.
Se mide en **pips** (1 pip = 0.0001 para la mayoría de pares FOREX).

In [ ]:
eurusd['range_pips'] = (eurusd['high'] - eurusd['low']) * 10000  # en pips

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(eurusd['range_pips'].dropna(), bins=60, color='darkorange', alpha=0.7, edgecolor='white')
axes[0].axvline(eurusd['range_pips'].median(), color='red', linestyle='--', label=f'Mediana: {eurusd["range_pips"].median():.1f} pips')
axes[0].set_title('Distribución del Rango Intra-Vela (H1)', fontsize=12)
axes[0].set_xlabel('Rango (pips)')
axes[0].legend()

# Rolling volatility (20 periodos)
eurusd['rolling_vol'] = eurusd['log_return'].rolling(20).std() * np.sqrt(252*24)
axes[1].plot(eurusd.index, eurusd['rolling_vol'], color='crimson', linewidth=0.8)
axes[1].set_title('Volatilidad Anualizada (ventana 20H)', fontsize=12)
axes[1].set_ylabel('Volatilidad')

plt.tight_layout()
plt.show()

## 5. Indicadores técnicos

Los indicadores técnicos son **features derivadas** de las series de precio OHLCV.
Se utilizan como variables de entrada para los modelos de Machine Learning.

### Familias de indicadores
| Familia | Indicadores | Mide |
|---------|------------|------|
| Medias Móviles | SMA(20,50,200), EMA(12,26,50) | Tendencia |
| Osciladores | RSI(14), Estocástico(K,D) | Sobrecompra/sobreventa |
| Volatilidad | ATR(14), Bandas de Bollinger(20) | Amplitud de movimiento |
| Momentum | MACD(12,26,9) | Fuerza y dirección del impulso |

In [ ]:
# Cargar indicadores técnicos
ind_path = INDICATORS_DIR / 'technical_indicator_eurusd.csv'
indicators = pd.read_csv(ind_path)
indicators['datetime'] = pd.to_datetime(indicators['timestamp'])
indicators = indicators.set_index('datetime').sort_index()

print(f'Indicadores EURUSD: {len(indicators)} filas')
print(f'Columnas: {list(indicators.columns)}')
print(f'\nTimeframes disponibles: {indicators["timeframe"].unique()}')
print(f'Valores nulos por columna (top 10):')
indicators.isnull().sum().sort_values(ascending=False).head(10)

In [ ]:
# Filtrar solo H1 para visualización
ind_h1 = indicators[indicators['timeframe'] == 'H1'].copy()

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Precio + SMAs
axes[0].plot(ind_h1.index, ind_h1['close'], label='Close', color='steelblue', linewidth=0.8)
axes[0].plot(ind_h1.index, ind_h1['sma_20'], label='SMA(20)', color='orange', linewidth=0.7, alpha=0.8)
axes[0].plot(ind_h1.index, ind_h1['sma_50'], label='SMA(50)', color='green', linewidth=0.7, alpha=0.8)
axes[0].plot(ind_h1.index, ind_h1['sma_200'], label='SMA(200)', color='red', linewidth=0.7, alpha=0.8)
axes[0].set_title('EURUSD H1 — Precio y Medias Móviles', fontsize=12)
axes[0].legend(loc='upper left')

# RSI
axes[1].plot(ind_h1.index, ind_h1['rsi_14'], color='purple', linewidth=0.8)
axes[1].axhline(70, color='red', linestyle='--', alpha=0.5, label='Sobrecompra (70)')
axes[1].axhline(30, color='green', linestyle='--', alpha=0.5, label='Sobreventa (30)')
axes[1].set_title('RSI(14)', fontsize=12)
axes[1].set_ylim(0, 100)
axes[1].legend(loc='upper left')

# MACD
axes[2].plot(ind_h1.index, ind_h1['macd_line'], label='MACD', color='blue', linewidth=0.8)
axes[2].plot(ind_h1.index, ind_h1['macd_signal'], label='Signal', color='red', linewidth=0.8)
axes[2].bar(ind_h1.index, ind_h1['macd_histogram'], color='gray', alpha=0.4, width=0.03, label='Histograma')
axes[2].axhline(0, color='black', linewidth=0.5)
axes[2].set_title('MACD(12,26,9)', fontsize=12)
axes[2].legend(loc='upper left')

plt.tight_layout()
plt.show()

## 6. Correlaciones entre indicadores

Analizar la **correlación** entre indicadores ayuda a:
- Identificar redundancias (features que miden lo mismo)
- Seleccionar combinaciones informativas para los modelos

In [ ]:
indicator_cols = ['rsi_14', 'atr_14', 'macd_line', 'macd_histogram',
                  'stoch_k', 'stoch_d', 'bb_upper_20', 'bb_lower_20']
available_cols = [c for c in indicator_cols if c in ind_h1.columns]

corr_matrix = ind_h1[available_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, ax=ax, vmin=-1, vmax=1)
ax.set_title('Correlación entre Indicadores Técnicos (EURUSD H1)', fontsize=13)
plt.tight_layout()
plt.show()

## 7. Comparación entre pares

Los 8 pares FOREX incluidos permiten evaluar la generalización de estrategias
a través de diferentes mercados con características distintas.

In [ ]:
# Cargar todos los pares disponibles
pairs = {}
for f in sorted(RATES_DIR.glob('*_h1_rates.csv')):
    symbol = f.stem.replace('_h1_rates', '')
    df = load_rates(symbol, 'h1')
    if not df.empty:
        df['log_return'] = np.log(df['close'] / df['close'].shift(1))
        pairs[symbol.upper()] = df

print(f'Pares cargados: {list(pairs.keys())}')

# Comparar volatilidades
vol_stats = pd.DataFrame({
    symbol: {
        'Media retorno (bps)': df['log_return'].mean() * 10000,
        'Volatilidad (H1)': df['log_return'].std() * 10000,
        'Rango medio (pips)': (df['high'] - df['low']).mean() * (10000 if 'jpy' not in symbol.lower() else 100),
        'Velas': len(df)
    }
    for symbol, df in pairs.items()
}).T

vol_stats.round(2)

## 8. Conclusiones del EDA

### Hallazgos principales

1. **Distribución de retornos**: Colas más pesadas que una normal (leptokurtosis),
   lo que justifica el uso de técnicas robustas frente a eventos extremos.

2. **Agrupamiento de volatilidad**: Periodos de alta volatilidad se agrupan temporalmente,
   lo que es relevante para la gestión de riesgo.

3. **Indicadores técnicos**: Proporcionan features multi-dimensionales derivadas del precio.
   La evaluación automatizada de cientos de combinaciones es más rigurosa que la selección manual.

4. **Diversificación entre pares**: Los 8 pares FOREX presentan volatilidades y comportamientos
   distintos, permitiendo evaluar la robustez de las estrategias.

> **Siguiente paso**: En el notebook `02_signal_discovery.ipynb` se analiza cómo las combinaciones
> de indicadores se evalúan estadísticamente para identificar señales con ventaja predictiva.